# 🐘🎓 Spark & HDFS: Пошаговое руководство

Этот ноутбук создан, чтобы научить вас работать с **HDFS** и **Apache Spark** в интерактивном режиме.

## ⚙️ Настройка окружения
Выполните эту ячейку, чтобы добавить команды Hadoop в систему.

In [15]:
import os

# Путь к Hadoop
hadoop_home = "/home/jovyan/work/hadoop"
hadoop_bin = hadoop_home + "/bin"
hadoop_conf = hadoop_home + "/etc/hadoop"

# Путь к Java (найден в контейнере)
java_home = "/usr/lib/jvm/java-17-openjdk-arm64"

# Добавляем переменные окружения
# %env - это магия IPython, которая устанавливает переменные для системных вызовов (!)
%env HADOOP_HOME={hadoop_home}
%env HADOOP_CONF_DIR={hadoop_conf}
%env JAVA_HOME={java_home}
%env PATH={hadoop_bin}:{os.environ['PATH']}

print(f"✅ Environment congigured!")
print(f"   HADOOP_HOME: {hadoop_home}")
print(f"   JAVA_HOME:   {java_home}")

env: HADOOP_HOME=/home/jovyan/work/hadoop
env: HADOOP_CONF_DIR=/home/jovyan/work/hadoop/etc/hadoop
env: JAVA_HOME=/usr/lib/jvm/java-17-openjdk-arm64
env: PATH=/home/jovyan/work/hadoop/bin:/home/jovyan/work/hadoop/bin:/home/jovyan/work/hadoop/bin:/home/jovyan/work/hadoop/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/spark/bin
✅ Environment congigured!
   HADOOP_HOME: /home/jovyan/work/hadoop
   JAVA_HOME:   /usr/lib/jvm/java-17-openjdk-arm64


Теперь проверим, что команда `hdfs` работает:

In [16]:
!hdfs version

Hadoop 3.3.6
Source code repository https://github.com/apache/hadoop.git -r 1be78238728da9266a4f88195058f08fd012bf9c
Compiled by ubuntu on 2023-06-18T08:22Z
Compiled on platform linux-x86_64
Compiled with protoc 3.7.1
From source with checksum 5652179ad55f76cb287d9c633bb53bbd
This command was run using /home/jovyan/work/hadoop/share/hadoop/common/hadoop-common-3.3.6.jar


### ⚠️ Если ячейка выше зависла со знаком `[*]`:
Это значит, что Jupyter не смог найти Java. Обычно ячейка должна отработать за 1-2 секунды.
Если это произошло, нажмите **Kernel -> Restart Kernel** в меню сверху и попробуйте снова.

---

## Шаг 1: Работа с HDFS (Консоль)

В Jupyter можно выполнять bash-команды, используя восклицательный знак `!`.
Давайте посмотрим, что у нас есть в файловой системе HDFS.

In [17]:
!hdfs dfs -ls /

2026-02-08 20:54:16,546 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxr-xr-x   - root supergroup          0 2026-02-08 19:45 /test_dir
drwxr-xr-x   - root supergroup          0 2026-02-08 20:03 /user


In [18]:
# Проверим нашу папку пользователя
!hdfs dfs -ls /user/myself/

2026-02-08 20:54:19,696 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   1 root supergroup   47673370 2026-02-08 20:00 /user/myself/yellow_tripdata_2023-01.parquet


---

## Шаг 2: Инициализация Spark

Создаем `SparkSession`.

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, desc

# Создаем сессию
spark = SparkSession.builder \
    .appName("SparkHDFSTutorial") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
spark

Spark Version: 3.5.0


---

## Шаг 3: Чтение данных (Data Loading)

Читаем Parquet файл.

In [20]:
# Читаем файл из HDFS
file_path = "hdfs://namenode:9000/user/myself/yellow_tripdata_2023-01.parquet"
df = spark.read.parquet(file_path)

# Выведем схему данных (типы колонок)
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



---

## Шаг 4: Основные операции с DataFrame

In [21]:
# Выбор конкретных колонок (Select)
df.select("tpep_pickup_datetime", "passenger_count", "total_amount").show(5)

+--------------------+---------------+------------+
|tpep_pickup_datetime|passenger_count|total_amount|
+--------------------+---------------+------------+
| 2023-01-01 00:32:10|            1.0|        14.3|
| 2023-01-01 00:55:08|            1.0|        16.9|
| 2023-01-01 00:25:04|            1.0|        34.9|
| 2023-01-01 00:03:48|            0.0|       20.85|
| 2023-01-01 00:10:29|            1.0|       19.68|
+--------------------+---------------+------------+
only showing top 5 rows



In [22]:
# Фильтрация (Filter / Where)
# Найдем поездки, где стоимость больше $100
expensive_trips = df.filter(col("total_amount") > 100)

print(f"Количество дорогих поездок: {expensive_trips.count()}")
expensive_trips.select("tpep_pickup_datetime", "total_amount").show(5)

Количество дорогих поездок: 36001
+--------------------+------------+
|tpep_pickup_datetime|total_amount|
+--------------------+------------+
| 2023-01-01 00:29:33|      107.87|
| 2023-01-01 00:51:35|      103.55|
| 2023-01-01 00:19:10|      189.98|
| 2023-01-01 00:31:05|      100.33|
| 2023-01-01 00:31:00|      115.37|
+--------------------+------------+
only showing top 5 rows



In [23]:
# Группировка и Агрегация (GroupBy)
# Посчитаем средний чек для каждого VendorID
df.groupBy("VendorID") \
  .agg(
      count("*").alias("total_trips"),
      avg("total_amount").alias("avg_fare")
  ) \
  .show()

+--------+-----------+------------------+
|VendorID|total_trips|          avg_fare|
+--------+-----------+------------------+
|       1|     827367|25.755021350872866|
|       2|    2239399| 27.48788289622012|
+--------+-----------+------------------+



---

## Шаг 5: Использование SQL

In [25]:
df.createOrReplaceTempView("taxi_trips")

In [26]:
# Пример SQL-запроса: Топ 5 самых дорогих поездок
spark.sql("""
    SELECT 
        tpep_pickup_datetime, 
        passenger_count, 
        trip_distance, 
        total_amount
    FROM taxi_trips
    ORDER BY total_amount DESC
    LIMIT 5
""").show()

+--------------------+---------------+-------------+------------+
|tpep_pickup_datetime|passenger_count|trip_distance|total_amount|
+--------------------+---------------+-------------+------------+
| 2023-01-24 12:43:44|            1.0|       177.88|      1169.4|
| 2023-01-09 16:17:32|            1.0|          0.0|      1000.0|
| 2023-01-30 13:17:33|            1.0|          0.0|       901.0|
| 2023-01-30 13:23:56|            1.0|          0.0|       751.0|
| 2023-01-30 16:17:35|            2.0|         8.81|       705.6|
+--------------------+---------------+-------------+------------+



---

## Шаг 6: Запись результатов обратно в HDFS

In [28]:
# Подготовим маленький DataFrame
summary_df = df.groupBy("payment_type").count()

# Сохраним его в CSV в HDFS
output_path = "hdfs://namenode:9000/user/myself/payment_summary"
summary_df.write.mode("overwrite").csv(output_path)

print("Данные сохранены!")

Данные сохранены!


In [29]:
# Проверяем
!hdfs dfs -ls /user/myself/payment_summary

2026-02-08 20:58:55,578 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 jovyan supergroup          0 2026-02-08 20:58 /user/myself/payment_summary/_SUCCESS
-rw-r--r--   3 jovyan supergroup         43 2026-02-08 20:58 /user/myself/payment_summary/part-00000-8f63b1da-d9e2-40a4-abaf-aa5f0ec92821-c000.csv


In [30]:
spark.stop()